In [1]:
# 设置工作目录
source("/home/user/data2/lit/bin/lit_utils.R")
source("/home/user/data3/lit/project/sORFs/sORFs.utils.R")
lib_text()

In [2]:
sep_add_info <- function(all_orfs,trans_mul_feature_path,gpe_path){
    # 提取染色体、起始位置、终止位置、链方向等信息
    matches <- str_match(all_orfs$ORF_id_trans, "([A-Z]+\\d+\\.\\d+[a-zA-Z_]*)([+-])(chr\\w+):(\\d+)-(\\d+)")
    all_orfs$ENS_id <-  matches[, 2]
    all_orfs$Strand <- matches[, 3]
    all_orfs$Chr <- matches[, 4]
    all_orfs$Start <- matches[, 5] %>% as.numeric()
    all_orfs$End <- matches[, 6] %>% as.numeric()

    # 加载基因名、转录本类型等信息并整合
    trans_mul_feature <- fread(trans_mul_feature_path)
    colnames(trans_mul_feature) <- c("Gene_ID", "Transcript_ID", "Transcript_type", "Gene_type", "Gene_name")
    merge(all_orfs, trans_mul_feature,
          by.x = "ENS_id", by.y = "Transcript_ID") -> all_orfs_1
    all_orfs_1 %>% as.data.frame() -> all_orfs_1

    # 注释小肽类型
    gpe <- fread(gpe_path, header = FALSE, data.table = FALSE)
    gpe[, c("V1", "V6", "V7")] -> gpe_selected
    colnames(gpe_selected) <- c("ENS_id", "CDS_start", "CDS_end")
    merge(all_orfs_1, gpe_selected) -> all_orfs_2
    get_orf_type_2(all_orfs_2) -> all_orfs_3
    return(all_orfs_3)
}

In [3]:
noncaono_sep_ms_supported_path <- "../processed/ms_res/noncano.sep.ms.supported.tab.info.txt"
noncano_sep_putative_path <- "/home/user/data3/lit/project/sORFs/01-ribo-seq/analysis/Run_for_human_20250227/human_trans_database_20250324/output/trans_based_sorfs.txt"
trans_mul_feature_path <- "/home/user/data2/lit/project/ZNF271/data/annotation/Ensembl_106_Gencode_v41_Human_Transcript_stable_ID_version_Gene_stable_ID_version_Gene_name_Transcript_type_gene_type.txt"
gpe_path <- "/home/user/data2/lit/project/ZNF271/data/annotation/gencode.v41.annotation.10.gpe"

In [4]:
fread_c(noncaono_sep_ms_supported_path) -> noncaono_sep_ms_supported
fread(noncano_sep_putative_path) -> noncano_sep_putative

In [5]:
# 随机抽取数量相同的负对照
noncano_sep_putative %>% filter(!Seq %in% noncaono_sep_ms_supported$Seq) -> noncano_sep_ms_not_supported
set.seed(123)
sample(1:nrow(noncano_sep_ms_not_supported),nrow(noncaono_sep_ms_supported)) ->  sample_rows
noncano_sep_ms_not_supported[sample_rows,] -> noncano_sep_ms_not_supported_sampled

In [6]:
nrow(noncano_sep_ms_not_supported_sampled)

[1] 29163

In [7]:
sep_add_info(noncano_sep_ms_not_supported_sampled,trans_mul_feature_path,gpe_path)->noncano_sep_ms_not_supported_sampled_add_info

In [8]:
fwrite_c(noncano_sep_ms_not_supported_sampled_add_info,"../processed/tab_info/noncano.sep.ms.not.supported.tab.info.txt")